In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
from sklearn.preprocessing import LabelEncoder
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator

In [5]:
df = pd.read_csv("./heart_disease_uci.csv")
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63,Male,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,67,Male,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,67,Male,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,37,Male,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,41,Female,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,54,Female,asymptomatic,127.0,333.0,True,st-t abnormality,154.0,False,0.0,NaN,NaN,NaN,1
916,62,Male,typical angina,NaN,139.0,False,st-t abnormality,NaN,NaN,NaN,NaN,NaN,NaN,0
917,55,Male,asymptomatic,122.0,223.0,True,st-t abnormality,100.0,False,0.0,NaN,NaN,fixed defect,2
918,58,Male,asymptomatic,NaN,385.0,True,lv hypertrophy,NaN,NaN,NaN,NaN,NaN,NaN,0


In [6]:
# Exploratory data analysis
print("number of rows and features : ", df.shape)

number of rows and features :  (920, 14)


In [7]:
# Checking if there are any missing variables
df.isnull().sum()

age           0
sex           0
cp            0
trestbps     59
chol         30
fbs          90
restecg       2
thalach      55
exang        55
oldpeak      62
slope       309
ca          611
thal        486
num           0
dtype: int64

In [8]:
# Deleting the rows with the missing values
df = df.dropna(axis=0)

In [9]:
# Checking if there are any missing variables
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
num         0
dtype: int64

In [10]:
# Removing the not so important features
del df['ca']
del df['thal']
del df['slope']
del df['oldpeak']

In [11]:
df.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'num'],
      dtype='object')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 0 to 748
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       299 non-null    int64  
 1   sex       299 non-null    object 
 2   cp        299 non-null    object 
 3   trestbps  299 non-null    float64
 4   chol      299 non-null    float64
 5   fbs       299 non-null    object 
 6   restecg   299 non-null    object 
 7   thalach   299 non-null    float64
 8   exang     299 non-null    object 
 9   num       299 non-null    int64  
dtypes: float64(3), int64(2), object(5)
memory usage: 25.7+ KB


In [13]:
def str_features_to_numeric(data):
    # Transforms all string features of the df to numeric features
    
    # Determination categorical features
    categorical_columns = []
    numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    features = data.columns.values.tolist()
    for col in features:
        if data[col].dtype in numerics: continue
        categorical_columns.append(col)
    
    # Encoding categorical features
    for col in categorical_columns:
        if col in data.columns:
            le = LabelEncoder()
            le.fit(list(data[col].astype(str).values))
            data[col] = le.transform(list(data[col].astype(str).values))
    
    return data

In [14]:
df = str_features_to_numeric(df)
df[['trestbps','chol','thalach']] = df[['trestbps','chol','thalach']].astype(int)

#if it's int64 set it as int32
for column in df.columns:
    if df[column].dtype == 'int64':
        df[column] = df[column].astype('int32')

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 0 to 748
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   age       299 non-null    int32
 1   sex       299 non-null    int32
 2   cp        299 non-null    int32
 3   trestbps  299 non-null    int32
 4   chol      299 non-null    int32
 5   fbs       299 non-null    int32
 6   restecg   299 non-null    int32
 7   thalach   299 non-null    int32
 8   exang     299 non-null    int32
 9   num       299 non-null    int32
dtypes: int32(10)
memory usage: 14.0 KB


C:\Users\Raghunathan R\AppData\Local\Temp\ipykernel_27320\2270910170.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = le.transform(list(data[col].astype(str).values))
C:\Users\Raghunathan R\AppData\Local\Temp\ipykernel_27320\3363758144.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['trestbps','chol','thalach']] = df[['trestbps','chol','thalach']].astype(int)
C:\Users\Raghunathan R\AppData\Local\Temp\ipykernel_27320\3363758144.py:7: SettingWithCopyWarning: 
A value is trying to be se

In [15]:
model = BayesianModel([('age','trestbps'),('age','fbs'),('sex','trestbps'),('exang','trestbps'),('trestbps','num'),('fbs','num'),('num','restecg'),('num','thalach'),('num','chol')])

c:\Users\Raghunathan R\AppData\Local\Programs\Python\Python310\lib\site-packages\pgmpy\models\BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


In [16]:
# Fit the model using maximum likelihood estimation
model.fit(df, estimator=MaximumLikelihoodEstimator)


In [17]:
import joblib

joblib.dump(model, "model.pkl")

['model.pkl']